# Summary of Objects Created

This notebook sets up a complete Snowflake Intelligence environment with Cortex AI capabilities for documentation assistance.

## Account Configuration
- Enables Cortex cross-region functionality (AWS_US)

## Security & Access
- **Role**: `snowflake_intelligence_admin` with privileges to create warehouses, databases, integrations, and agents
- **User Configuration**: Assigns the admin role to current user and sets default warehouse

## Infrastructure
- **Warehouse**: `Snow_Intelligence_wh` (XSMALL, auto-suspend 300s, auto-resume enabled)
- **Database**: `snowflake_intelligence`
- **Schema**: `snowflake_intelligence.agents` (with agent creation privileges)

## AI Agent
- **Agent**: `Snowflake_Documentation_Assistant` - An intelligent documentation assistant that:
  - Uses Cortex Search to query official Snowflake documentation
  - Provides accurate answers about Snowflake features, syntax, and best practices
  - Includes sample questions for common use cases
  - Connected to the shared Snowflake documentation search service

## Key Features
- Complete role-based access control setup
- Optimized warehouse configuration for AI workloads  
- Ready-to-use documentation search capabilities
- Integrated with official Snowflake documentation corpus

This setup provides a production-ready environment for AI-powered Snowflake documentation assistance.notebook

Load Snowflake Documentation through the Snowflake Marketplace.
This is a knowledge based extension (please briefly explain)
website: https://app.snowflake.com/marketplace/listing/GZSTZ67BY9OQ4/snowflake-snowflake-documentation
leave the name alone. add public to the roles available



In [ ]:
use role accountadmin;
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US';

In [ ]:
use role accountadmin;
create role if not exists snowflake_intelligence_admin;
grant create warehouse on account to role snowflake_intelligence_admin;
grant create database on account to role snowflake_intelligence_admin;
grant create integration on account to role snowflake_intelligence_admin;

In [ ]:
use role accountadmin;

-- Create a dedicated warehouse for the demo with auto-suspend/resume
CREATE OR REPLACE WAREHOUSE Snow_Intelligence_wh 
    WITH WAREHOUSE_SIZE = 'XSMALL'
    AUTO_SUSPEND = 300
    AUTO_RESUME = TRUE;

-- Grant usage on warehouse to admin role
GRANT USAGE ON WAREHOUSE Snow_Intelligence_wh TO ROLE snowflake_intelligence_admin;

-- Grant usage to PUBLIC (needed for agent access later)
GRANT USAGE ON WAREHOUSE Snow_Intelligence_wh TO ROLE PUBLIC;

In [ ]:
-- If you want to use a variable for clarity/performance
set current_user_name = current_user();
grant role snowflake_intelligence_admin to user identifier($current_user_name);
alter user identifier($current_user_name) set default_role = snowflake_intelligence_admin;
alter user identifier($current_user_name) set default_warehouse = Snow_Intelligence_wh;

In [ ]:
use role snowflake_intelligence_admin;
create database if not exists snowflake_intelligence;
create schema if not exists snowflake_intelligence.agents;
grant create agent on schema snowflake_intelligence.agents to role snowflake_intelligence_admin;

In [ ]:
use role snowflake_intelligence_admin;

-- CREATES A SNOWFLAKE INTELLIGENCE AGENT WITH CORTEX SEARCH
CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Snowflake_Documentation_Assistant
WITH PROFILE='{ "display_name": "Snowflake Documentation Assistant" }'
    COMMENT=$$ An intelligent agent that helps users find answers to Snowflake questions by searching official Snowflake documentation. Provides accurate, up-to-date information on features, syntax, best practices, and troubleshooting. $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": "auto"
  },
  "instructions": {
    "response": "You are a helpful Snowflake documentation assistant with deep knowledge of Snowflake's features and capabilities. When users ask questions about Snowflake, search the official documentation to provide accurate, detailed answers. Always cite the source documentation when providing information. Format code examples clearly with proper syntax highlighting. If a user's question is unclear, ask for clarification. Provide step-by-step instructions when explaining procedures. Be concise but thorough in your explanations.",
    "orchestration": "Always search the Snowflake documentation first when users ask questions about Snowflake features, syntax, configuration, or troubleshooting. Use the search results to provide comprehensive answers backed by official documentation. If the search doesn't return relevant results, let the user know and suggest they rephrase their question or check if they're asking about a feature that might not be documented yet. When multiple documentation sources are relevant, reference the most appropriate ones for the user's specific question.",
    "sample_questions": [
      {
        "question": "How do I create a Cortex Search service in Snowflake?"
      },
      {
        "question": "What are the different types of Snowflake tables and when should I use each?"
      },
      {
        "question": "How do I set up data sharing between Snowflake accounts?"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Snowflake Documentation",
        "description": "Searches Snowflake's official documentation to find relevant information about Snowflake features, SQL syntax, configuration options, best practices, troubleshooting guides, and API references. Returns the most relevant documentation chunks with titles and source URLs."
      }
    }
  ],
  "tool_resources": {
    "Search Snowflake Documentation": {
      "id_column": "SOURCE_URL",
      "max_results": 5,
      "name": "SNOWFLAKE_DOCUMENTATION.SHARED.CKE_SNOWFLAKE_DOCS_SERVICE",
      "title_column": "DOCUMENT_TITLE"
    }
  }
}
$$;

In [ ]:
use role snowflake_intelligence_admin;

-- Ensure database and schema access for PUBLIC
GRANT USAGE ON DATABASE SNOWFLAKE_INTELLIGENCE TO ROLE PUBLIC;
GRANT USAGE ON SCHEMA SNOWFLAKE_INTELLIGENCE.AGENTS TO ROLE PUBLIC;

-- Missing: Grant usage on the specific agent
GRANT USAGE ON AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Snowflake_Documentation_Assistant TO ROLE PUBLIC;